Your Full Name: 
    
`Arindam Choudhury`

    Nutan Mandale
    
    Humberto Gonzalez Granda

Your Uplevel Email Address:
    
    arindam.choudhury.email@gmail.com
    
    nutan.mandale@gmail.com
    
    HumbertoGonzalezGranda@gmail.com

Name of the Problem Statement of Submission:
    
    ShopTalk (Project-6)

# Finetune Embeddings

We will be doing finetuning of embedding model from Sentence Transformer and see the results

There are three main sections:
1. Generate Synthetic Dataset with LLM (model used : `gpt-3.5-turbo`)
2. Finetuning the model (using `SentenceTransformers` model `BAAI/bge-small-en`)
3. Evaluating the model (Custom Evaluation with LlamaIndex)

### Generate Synthetic Dataset with LLM

First, we generate a synthetic dataset of (query, relevant documents) pairs from a corpus of documents without labelers by leveraging LLM.

In [1]:
import json
import pandas as pd
from tqdm import tqdm
import re
import os
import uuid
import shutil
from pathlib import Path
import boto3
from s3fs import S3FileSystem
from sklearn.model_selection import train_test_split
from llama_index.core import SimpleDirectoryReader
from llama_index.core.node_parser import JSONNodeParser
from llama_index.core.schema import MetadataMode
from llama_index.llms.openai import OpenAI
from llama_index.core import VectorStoreIndex
from llama_index.core.schema import TextNode
from dotenv import load_dotenv
load_dotenv()

True

##### File save and load function to S3 and from S3

In [2]:
def save_s3(file_path, object):
    fs = S3FileSystem()
    with fs.open(file_path, "w") as file:
        json.dump(object, file)

def load_s3(file_path):
    fs = S3FileSystem()
    with fs.open(file_path, "r") as file:
        obj = json.load(file)
    return obj

##### Define files path

In [3]:
ABO_BUCKET_NAME:      str = os.getenv("ABO_BUCKET_NAME")
YOUR_S3_BUCKET_NAME:  str = os.getenv("YOUR_S3_BUCKET_NAME")
ARTIFACTS_FOLDER:     str = os.getenv("ARTIFACTS_FOLDER")
WORKING_DIR:          str = os.getenv("WORKING_DIR")
EDA_FOLDER_NAME:      str = os.getenv("EDA_FOLDER_NAME")

DATASET_PATH          = f"s3://{YOUR_S3_BUCKET_NAME}/{EDA_FOLDER_NAME}/dataset.json"
FINETUNE_FILE_PATH    = f"s3://{YOUR_S3_BUCKET_NAME}/FINETUNE/finetune_files/"

FINETUNE_MODEL_LOCAL  = f"{WORKING_DIR}finetuned_model"
FINETUNE_MODEL_S3     = "FINETUNE/finetuned_model"

FINETUNE_EVAL_LOCAL   = f"{WORKING_DIR}evaluation/"
FINETUNE_EVAL_S3      = "FINETUNE/evaluation/"

In [4]:
TRAIN_SPLIT   = FINETUNE_FILE_PATH + "train/" + "train_split.json"
VAL_SPLIT     = FINETUNE_FILE_PATH + "val/" + "val_split.json"
TRAIN_CORPUS  = FINETUNE_FILE_PATH + "train_corpus.json"
VAL_CORPUS    = FINETUNE_FILE_PATH + "val_corpus.json"
TRAIN_QUERY   = FINETUNE_FILE_PATH + "train_queries.json"
VAL_QUERY     = FINETUNE_FILE_PATH + "train_relevant_docs.json"
TRAIN_DOCS    = FINETUNE_FILE_PATH + "val_queries.json"
VAL_DOCS      = FINETUNE_FILE_PATH + "val_relevant_docs.json"
TRAIN_DATASET = FINETUNE_FILE_PATH + "final_train_dataset.json"
VAL_DATASET   = FINETUNE_FILE_PATH + "final_val_dataset.json"

##### Load the main data file. Split into train and validation. (A sample of 100 json datapoints are taken in train and 20 in validation)

In [5]:
dataset = pd.read_json(DATASET_PATH) # Lets take 100 datapoints for fine tuning and 20 for validation
TRAIN_FILES, VAL_FILES = train_test_split(dataset, train_size = 100, test_size = 20, random_state = 100, shuffle=True)
TRAIN_FILES.to_json(TRAIN_SPLIT, orient='records')
VAL_FILES.to_json(VAL_SPLIT, orient='records')

##### Generate Corpus
First, we create the corpus of text chunks by leveraging LlamaIndex to load some json dataset, and parsing/chunking into one chunks.

In [6]:
def load_corpus(folder, verbose=False):

    s3_fs = S3FileSystem(anon=False, endpoint_url=None)
    input_dir = f"{YOUR_S3_BUCKET_NAME}/FINETUNE/finetune_files/{folder}"

    reader = SimpleDirectoryReader(
        input_dir=input_dir,
        fs=s3_fs,
        recursive=True
    )
        
    docs = reader.load_data()
    
    parser = JSONNodeParser()
    nodes = parser.get_nodes_from_documents(docs, show_progress=verbose)

    corpus = {node.node_id: node.get_content(metadata_mode=MetadataMode.NONE) for node in nodes}

    return corpus

In [7]:
train_corpus = load_corpus("train", verbose=True)
save_s3(TRAIN_CORPUS, train_corpus)

val_corpus   = load_corpus("val", verbose=True)
save_s3(VAL_CORPUS, val_corpus)

/Users/arindam/Machine Learning/GitHub_Repository/ShopChat_APP/venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Parsing nodes: 100%|██████████| 1/1 [00:00<00:00, 188.40it/s]


#### Generate synthetic queries
Now, we use an LLM (gpt-3.5-turbo) to generate questions using each text chunk in the corpus as context.

Each pair of (generated question, text chunk used as context) becomes a datapoint in the finetuning dataset (either for training or evaluation).

In [8]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [9]:
def generate_queries(
    corpus,
    num_questions_per_chunk=1,
    prompt_template=None,
    verbose=False,
):
    """
    Automatically generate hypothetical questions that could be answered with
    doc in the corpus.
    """
    llm = OpenAI(model='gpt-3.5-turbo')

    prompt_template = prompt_template or """\
    Context information is below.
    
    ---------------------
    {context_str}
    ---------------------
    
    Given the context information and not prior knowledge.
    generate only questions based on the below query.
    
    You are a buyer. Your task is to ask \
    {num_questions_per_chunk} questions with respect to buying this product. The questions should be from the context \
    document mostly on brand_in_en_us, bullet_point_in_en_us, color_in_en_us, fabric_type_in_en_us, item_keywords_in_en_us, \
    item_name_in_en_us and product_description_in_en_us. \
    Restrict the questions to the context information provided."
    """

    queries = {}
    relevant_docs = {}
    for node_id, text in tqdm(corpus.items()):
        query = prompt_template.format(context_str=text, num_questions_per_chunk=num_questions_per_chunk)
        response = llm.complete(query)
 
        result = str(response).strip().split("\n")
        questions = [
            re.sub(r"^\d+[\).\s]", "", question).strip() for question in result
        ]
        questions = [question for question in questions if len(question) > 0]
        
        for question in questions:
            question_id = str(uuid.uuid4())
            queries[question_id] = question
            relevant_docs[question_id] = [node_id]
    return queries, relevant_docs

In [10]:
train_queries, train_relevant_docs = generate_queries(train_corpus)
val_queries, val_relevant_docs     = generate_queries(val_corpus)

100%|██████████| 20/20 [00:18<00:00,  1.10it/s]


In [11]:
save_s3(TRAIN_QUERY, train_queries)
save_s3(TRAIN_DOCS, train_relevant_docs)
save_s3(VAL_QUERY, val_queries)
save_s3(VAL_DOCS, val_relevant_docs)

#### Merge data 
Finally, we do some minor re-organization to make it easier to access the dataset for training and evaluation.

In [12]:
train_dataset = {
    'queries': train_queries,
    'corpus': train_corpus,
    'relevant_docs': train_relevant_docs,
}

val_dataset = {
    'queries': val_queries,
    'corpus': val_corpus,
    'relevant_docs': val_relevant_docs,
}

In [13]:
save_s3(TRAIN_DATASET, train_dataset)
save_s3(VAL_DATASET, val_dataset)

## Fine tuning
Finetune an opensource sentencetransformers embedding model with synthetically generated dataset.

#### Load pretrained model

In [14]:
from sentence_transformers import SentenceTransformer

In [15]:
model_id = "BAAI/bge-small-en"
model = SentenceTransformer(model_id)

/Users/arindam/Machine Learning/GitHub_Repository/ShopChat_APP/venv/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [16]:
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': True}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

#### Define dataloader

In [17]:
from torch.utils.data import DataLoader
from sentence_transformers import InputExample

In [18]:
train_dataset = load_s3(TRAIN_DATASET)
val_dataset   = load_s3(VAL_DATASET)

In [19]:
BATCH_SIZE = 10

corpus = train_dataset['corpus']
queries = train_dataset['queries']
relevant_docs = train_dataset['relevant_docs']

examples = []
for query_id, query in queries.items():
    node_id = relevant_docs[query_id][0]
    text = corpus[node_id]
    example = InputExample(texts=[query, text])
    examples.append(example)

In [20]:
loader = DataLoader(
    examples, batch_size=BATCH_SIZE
)

#### Define loss
MultipleNegativesRankingLoss is a great loss function if you only have positive pairs, for example, only pairs of similar texts like pairs of paraphrases, pairs of duplicate questions, pairs of (query, response), or pairs of (source_language, target_language).

This loss function works great to train embeddings for retrieval setups where you have positive pairs (e.g. (query, relevant_doc)) as it will sample in each batch n-1 negative docs randomly.

The performance usually increases with increasing batch sizes.

In [21]:
from sentence_transformers import losses

In [22]:
loss = losses.MultipleNegativesRankingLoss(model)

#### Define evaluator
We setup an evaluator with our val split of the dataset to monitor how well the embedding model is performing during training.

In [23]:
from sentence_transformers.evaluation import InformationRetrievalEvaluator

In [24]:
corpus = val_dataset['corpus']
queries = val_dataset['queries']
relevant_docs = val_dataset['relevant_docs']

evaluator = InformationRetrievalEvaluator(queries, corpus, relevant_docs)

#### Run training
The training loop is very straight forward to steup thanks to sentencetransformers' high-level model training API. All we need to do is plugging in the data loader, loss function, and evaluator that we defined in the previous cells (along with a couple of additional minor settings).

In [25]:
# We train the model for very few epochs.
# This should typically be higher for better performance.
EPOCHS = 2

In [26]:
warmup_steps = int(len(loader) * EPOCHS * 0.1)

model.fit(
    train_objectives=[(loader, loss)],
    epochs=EPOCHS,
    warmup_steps=warmup_steps,
    output_path=FINETUNE_MODEL_LOCAL,
    save_best_model=True,
    show_progress_bar=True,
    evaluator=evaluator, 
    evaluation_steps=20,
)

Epoch: 100%|██████████| 2/2 [00:10<00:00,  5.06s/it]


##### Save the finetuned model in S3... But we need to save this to local from before saving into S3

In [27]:
# Function to upload directory to S3
def upload_directory_to_s3(bucket_name, s3_folder, local_directory):
    s3_client = boto3.client('s3')
    for root, dirs, files in os.walk(local_directory):
        for file in files:
            local_path = os.path.join(root, file)
            relative_path = os.path.relpath(local_path, local_directory)
            s3_path = os.path.join(s3_folder, relative_path)
            
            s3_client.upload_file(local_path, bucket_name, s3_path)
            print(f"Uploaded {s3_path} to S3 bucket {bucket_name}")

In [28]:
# Call the function to upload the model directory to S3
upload_directory_to_s3(YOUR_S3_BUCKET_NAME, FINETUNE_MODEL_S3, FINETUNE_MODEL_LOCAL)

Uploaded FINETUNE/finetuned_model/model.safetensors to S3 bucket shopchat-s3-buckect
Uploaded FINETUNE/finetuned_model/tokenizer_config.json to S3 bucket shopchat-s3-buckect
Uploaded FINETUNE/finetuned_model/special_tokens_map.json to S3 bucket shopchat-s3-buckect
Uploaded FINETUNE/finetuned_model/config.json to S3 bucket shopchat-s3-buckect
Uploaded FINETUNE/finetuned_model/config_sentence_transformers.json to S3 bucket shopchat-s3-buckect
Uploaded FINETUNE/finetuned_model/tokenizer.json to S3 bucket shopchat-s3-buckect
Uploaded FINETUNE/finetuned_model/README.md to S3 bucket shopchat-s3-buckect
Uploaded FINETUNE/finetuned_model/sentence_bert_config.json to S3 bucket shopchat-s3-buckect
Uploaded FINETUNE/finetuned_model/vocab.txt to S3 bucket shopchat-s3-buckect
Uploaded FINETUNE/finetuned_model/modules.json to S3 bucket shopchat-s3-buckect
Uploaded FINETUNE/finetuned_model/1_Pooling/config.json to S3 bucket shopchat-s3-buckect
Uploaded FINETUNE/finetuned_model/eval/Information-Retrie

## Custom Evaluation with LlamaIndex
We evaluate 3 different embedding models:

1. GeminiEmbedding(model_name = "models/embedding-001")
2. open source BAAI/bge-small-en from HuggungFace and
3. our `finetuned` embedding model.

We consider 2 evaluation approaches:

1. a simple custom hit rate metric
2. using InformationRetrievalEvaluator from sentence_transformers

We show that finetuning on synthetic (LLM-generated) dataset significantly improve upon an opensource embedding model.

#### Load data

First, let's load the synthetic dataset we automatically generated from our corpus (without having access to any labellers).

In [29]:
train_dataset = load_s3(TRAIN_DATASET)
val_dataset   = load_s3(VAL_DATASET)

#### Define eval function

##### Option 1: We use a simple hit rate metric for evaluation:

* for each (query, relevant_doc) pair,

* we retrieve top-k documents with the query, and

* it's a hit if the results contain the relevant_doc.

In [30]:
def evaluate(
    dataset,
    embed_model,
    top_k=5,
    verbose=False,
):
    corpus = dataset['corpus']
    queries = dataset['queries']
    relevant_docs = dataset['relevant_docs']

    nodes = [TextNode(id_=id_, text=text) for id_, text in corpus.items()] 
    index = VectorStoreIndex(
        nodes, 
        embed_model=embed_model, 
        show_progress=True
    )
    retriever = index.as_retriever(similarity_top_k=top_k)

    eval_results = []
    for query_id, query in tqdm(queries.items()):
        retrieved_nodes = retriever.retrieve(query)
        retrieved_ids = [node.node.node_id for node in retrieved_nodes]
        expected_id = relevant_docs[query_id][0]
        is_hit = expected_id in retrieved_ids  # assume 1 relevant doc
        
        eval_result = {
            'is_hit': is_hit,
            'retrieved': retrieved_ids,
            'expected': expected_id,
            'query': query_id,
        }
        eval_results.append(eval_result)
    return eval_results

##### Option 2: We use the InformationRetrievalEvaluator from sentence_transformers.

This provides a more comprehensive suite of metrics, but we can only run it against the sentencetransformers compatible models (open source and our finetuned model, not the OpenAI embedding model).

In [31]:
from sentence_transformers.evaluation import InformationRetrievalEvaluator
from sentence_transformers import SentenceTransformer

def evaluate_st(
    dataset,
    model_id,
    name,
):
    corpus = dataset['corpus']
    queries = dataset['queries']
    relevant_docs = dataset['relevant_docs']

    evaluator = InformationRetrievalEvaluator(queries, corpus, relevant_docs, name=name)
    model = SentenceTransformer(model_id)
    output_path = FINETUNE_EVAL_LOCAL
    Path(output_path).mkdir(exist_ok=True, parents=True)
    return evaluator(model, output_path=output_path)

#### RUN Evals

##### Open AI

In [32]:
from llama_index.embeddings.openai import OpenAIEmbedding
ada = OpenAIEmbedding()

In [33]:
ada_val_results = evaluate(val_dataset, ada)

100%|██████████| 20/20 [00:04<00:00,  4.85it/s]


In [34]:
df_ada = pd.DataFrame(ada_val_results)

In [35]:
hit_rate_ada = df_ada['is_hit'].mean()
hit_rate_ada

1.0

##### BAAI/bge-small-en

In [36]:
bge = "local:BAAI/bge-small-en"
bge_val_results = evaluate(val_dataset, bge)

/Users/arindam/Machine Learning/GitHub_Repository/ShopChat_APP/venv/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
100%|██████████| 20/20 [00:00<00:00, 21.58it/s]


In [37]:
df_bge = pd.DataFrame(bge_val_results)

In [38]:
hit_rate_bge = df_bge['is_hit'].mean()
hit_rate_bge

1.0

In [39]:
evaluate_st(val_dataset, "BAAI/bge-small-en", name='bge')

0.975

##### Finetuned Model

In [40]:
finetuned = f"local:{FINETUNE_MODEL_LOCAL}"
val_results_finetuned = evaluate(val_dataset, finetuned)

100%|██████████| 20/20 [00:00<00:00, 22.74it/s]


In [41]:
df_finetuned = pd.DataFrame(val_results_finetuned)

In [42]:
evaluate_st(val_dataset, FINETUNE_MODEL_LOCAL, name='finetuned')

0.975

In [43]:
hit_rate_finetuned = df_finetuned['is_hit'].mean()
hit_rate_finetuned

1.0

#### Summary of Results

##### Hit Rate

In [44]:
df_ada['model'] = 'ada'
df_bge['model'] = 'bge'
df_finetuned['model'] = 'fine_tuned'

###### We can see that fine-tuning our small open-source embedding model drastically improve its retrieval quality (even approaching the quality of the proprietary OpenAI embedding)!

In [45]:
df_all = pd.concat([df_ada, df_bge, df_finetuned])
df_all.groupby('model').mean('is_hit')

,is_hit
model,
ada,1.0
bge,1.0
fine_tuned,1.0


##### InformationRetrievalEvaluator

In [46]:
df_st_bge = pd.read_csv(FINETUNE_EVAL_LOCAL + 'Information-Retrieval_evaluation_bge_results.csv')
df_st_finetuned = pd.read_csv(FINETUNE_EVAL_LOCAL + 'Information-Retrieval_evaluation_finetuned_results.csv')

###### We can see that embedding finetuning improves metrics consistently across the suite of eval metrics

In [47]:
df_st_bge['model'] = 'bge'
df_st_finetuned['model'] = 'fine_tuned'
df_st_all = pd.concat([df_st_bge, df_st_finetuned])
df_st_all = df_st_all.set_index('model')
df_st_all

,epoch,steps,cos_sim-Accuracy@1,cos_sim-Accuracy@3,cos_sim-Accuracy@5,cos_sim-Accuracy@10,cos_sim-Precision@1,cos_sim-Recall@1,cos_sim-Precision@3,cos_sim-Recall@3,...,dot_score-Recall@1,dot_score-Precision@3,dot_score-Recall@3,dot_score-Precision@5,dot_score-Recall@5,dot_score-Precision@10,dot_score-Recall@10,dot_score-MRR@10,dot_score-NDCG@10,dot_score-MAP@100
model,,,,,,,,,,,,,,,,,,,,,
bge,-1,-1,0.95,1.0,1.0,1.0,0.95,0.95,0.333333,1.0,...,0.95,0.333333,1.0,0.2,1.0,0.1,1.0,0.975,0.981546,0.975
fine_tuned,-1,-1,0.95,1.0,1.0,1.0,0.95,0.95,0.333333,1.0,...,0.95,0.333333,1.0,0.2,1.0,0.1,1.0,0.975,0.981546,0.975


In [48]:
# Call the function to upload evaluation results to S3
upload_directory_to_s3(YOUR_S3_BUCKET_NAME, FINETUNE_EVAL_S3, FINETUNE_EVAL_LOCAL)

Uploaded FINETUNE/evaluation/Information-Retrieval_evaluation_bge_results.csv to S3 bucket shopchat-s3-buckect
Uploaded FINETUNE/evaluation/Information-Retrieval_evaluation_finetuned_results.csv to S3 bucket shopchat-s3-buckect


In [49]:
if os.path.exists(WORKING_DIR):
    shutil.rmtree(WORKING_DIR)
    print("finetune folder removed successfully.")

finetune folder removed successfully.
